In [15]:
from typing import Any, Dict, List
import os
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
load_dotenv()

True

In [16]:
def refund_flight(flight_PNR: str) -> str:
    return f"Refunded Flight with PNR {flight_PNR}"

In [ ]:
api_key = os.getenv("OPEN_ROUTER_API_KEY") 

model_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    api_key=api_key,
    model="qwen/qwen3-vl-30b-a3b-thinking",    
    model_info={
        "family": "qwen",
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "structured_output": True
    },
)

In [18]:
travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

In [19]:
flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight PNR numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

In [20]:
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)

In [21]:
task = 'I want to refund my flight'

async def run_team_stream() -> None:

    task_result = await Console(team.run_stream(task = task))

    last_message = task_result.messages[-1]


    while (isinstance(last_message,HandoffMessage) and last_message.target == 'user'):

        user_ka_message = input("User : ")

        task_result = await Console(team.run_stream(task = HandoffMessage(source='user',target=last_message.source,content=user_ka_message)))

        last_message = task_result.messages[-1]

await run_team_stream()

---------- TextMessage (user) ----------
I want to refund my flight


---------- ToolCallRequestEvent (travel_agent) ----------
[FunctionCall(id='call_d65559ebcf0849aa998358', arguments=' {}', name='transfer_to_flights_refunder')]
---------- ToolCallExecutionEvent (travel_agent) ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', name='transfer_to_flights_refunder', call_id='call_d65559ebcf0849aa998358', is_error=False)]
---------- HandoffMessage (travel_agent) ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
---------- ThoughtEvent (flights_refunder) ----------
To process your flight refund, I'll need your flight PNR (Booking Reference) number. Could you please provide this number? Once I have it, I'll initiate the refund immediately.  


---------- ToolCallRequestEvent (flights_refunder) ----------
[FunctionCall(id='call_fbafd41e1f1c41d388b86c', arguments=' {}', name='transfer_to_user')]
---------- ToolCallExecutionEvent (flights